In [29]:
import pandas as pd

In [30]:
propostas = pd.read_csv('application_record.csv')
pg = pd.read_csv('pagamentos_largo.csv')

In [4]:
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

In [17]:
pg['default'] = pg.loc[:, 'mes_00':'mes_24'].isin(['C']).any(axis=1)

# Ver as primeiras linhas para conferir
print(pg.head())

        ID mes_0 mes_1 mes_10 mes_11 mes_12 mes_13 mes_14 mes_15 mes_16  ...  \
0  5001718     0     0      0      0      0      0      0      0    NaN  ...   
1  5001719     0     0      C      C      C      C      C      C      C  ...   
2  5001720     0     0      0      0      0      0      0      0      0  ...   
3  5001723     0     0    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
4  5001726     0     0      C      C      C      C      C      C      C  ...   

  mes_23 mes_24 mes_3 mes_4 mes_5 mes_6 mes_7 mes_8 mes_9 default  
0      0    NaN     0     0     0     0   NaN     0   NaN   False  
1      C      C     C     C     C     C     C     C     C    True  
2      0      0     0     0     0     0     0     0     0   False  
3    NaN    NaN     0     0     0     0     0   NaN   NaN   False  
4      C      C     0     0     0     C     C     C     C    True  

[5 rows x 27 columns]


#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

In [19]:
# Criar uma coluna 'default_total' que conta o número de meses em default (considerando 'C' como default)
pg['default_total'] = pg.loc[:, 'mes_0':'mes_24'].isin(['C']).sum(axis=1)

# Criar uma coluna 'tem_default' que marca True para clientes com pelo menos 1 mês em default
pg['tem_default'] = pg['default_total'] > 0

# Exibir os resultados
resultado = pg['tem_default'].value_counts()
print(resultado)

tem_default
True     13516
False     7421
Name: count, dtype: int64


#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

In [25]:
# Selecionar somente os clientes da base de propostas que estão na base de pagamentos
clientes_com_cartao = propostas[propostas['ID'].isin(pg['ID'])]

# Filtrar os clientes na base de pagamentos para incluir apenas aqueles da base de propostas
pg_filtrado = pg[pg['ID'].isin(clientes_com_cartao['ID'])].copy()

# Criar uma coluna 'default_total' que conta o número de meses em default (considerando 'C' como default)
pg_filtrado.loc[:, 'default_total'] = pg_filtrado.loc[:, 'mes_0':'mes_24'].isin(['C']).sum(axis=1)

# Criar uma coluna 'tem_default' que marca True para clientes com pelo menos 1 mês em default
pg_filtrado.loc[:, 'tem_default'] = pg_filtrado['default_total'] > 0

# Contar o número de clientes com e sem default
resultado = pg_filtrado['tem_default'].value_counts()
print(resultado)

tem_default
True     10543
False     6107
Name: count, dtype: int64


#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

In [26]:
# Selecionar somente os clientes da base de propostas que estão na base de pagamentos e criar coluna 'tem_default'
clientes_com_cartao = propostas[propostas['ID'].isin(pg['ID'])]
pg_filtrado = pg[pg['ID'].isin(clientes_com_cartao['ID'])].copy()

# Criar coluna 'default_total' e 'tem_default'
pg_filtrado.loc[:, 'default_total'] = pg_filtrado.loc[:, 'mes_0':'mes_24'].isin(['C']).sum(axis=1)
pg_filtrado.loc[:, 'tem_default'] = pg_filtrado['default_total'] > 0

# Consolidar as informações em uma única linha de código usando merge
dados_consolidados = pd.merge(clientes_com_cartao, pg_filtrado[['ID', 'tem_default']], on='ID')

print(dados_consolidados.head())

        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008806           M            Y               Y             0   
1  5008810           F            N               Y             0   
2  5008811           F            N               Y             0   
3  5112956           M            Y               Y             0   
4  5008825           F            Y               N             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          112500.0               Working  Secondary / secondary special   
1          270000.0  Commercial associate  Secondary / secondary special   
2          270000.0  Commercial associate  Secondary / secondary special   
3          270000.0               Working               Higher education   
4          130500.0               Working              Incomplete higher   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
0               Married  House / apartment      -21474 

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [28]:
# Contar os valores na coluna 'tem_default'
contagem_default = dados_consolidados['tem_default'].value_counts()
print(contagem_default)

tem_default
True     10543
False     6107
Name: count, dtype: int64
